In [5]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
import re # For entity extraction
import random

# --- 1. Preprocessing ---
nltk.download('punkt', quiet=True)
nltk.download('punkt_tab', quiet=True)
nltk.download('stopwords', quiet=True)
nltk.download('wordnet', quiet=True)

lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

In [2]:
def preprocess_text(text):
    """Applies preprocessing (tokenization, stopword removal, lemmatization)."""
    tokens = nltk.word_tokenize(text.lower())
    processed_tokens = [
        lemmatizer.lemmatize(token) for token in tokens
        if token.isalnum() and token not in stop_words
    ]
    return " ".join(processed_tokens)

In [6]:
# --- 2. Load Data and Train Intent Classifier ---
df = pd.read_csv('customer_support.csv')

# Create a 'processed_query' column
df['processed_query'] = df['Example Query'].apply(preprocess_text)

# We use a Pipeline to combine TF-IDF and a Classifier
# This is our "Intent Recognition" model [cite: 37]
intent_classifier = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', MultinomialNB()) # A simple, fast classifier
])

# Train the model
X_train = df['processed_query']
y_train = df['Intent']
intent_classifier.fit(X_train, y_train)

print("Bot training complete.")

# --- 3. Store Responses ---
# Create a dictionary to hold our rule-based responses [cite: 40]
responses = {}
for intent, group in df.groupby('Intent'):
    # Store all possible responses for an intent
    responses[intent] = list(group['Response'])

Bot training complete.


In [7]:
def extract_order_number(text):
    """Extracts an order number (e.g., #12345 or 12345) using regex."""
    # Matches a # symbol (optional) followed by 5 or more digits
    match = re.search(r'#?(\d{5,})', text)
    if match:
        return match.group(1) # Return just the number
    return None

In [8]:
def get_bot_response(user_query):
    """Gets a full response from the chatbot."""

    # 1. Preprocess query
    processed_query = preprocess_text(user_query)

    # 2. Classify intent
    intent = intent_classifier.predict([processed_query])[0]

    # 3. Extract entities
    order_num = extract_order_number(user_query)

    # 4. Provide the corresponding response [cite: 51]

    # Get a random response from the list for that intent
    response_template = random.choice(responses.get(intent, ["Sorry, I don't understand that."]))

    # --- Response Formatting ---

    # If the intent is Order Status, try to fill in the order number
    if intent == 'Order Status':
        if order_num:
            final_response = response_template.format(order_num=order_num)
        else:
            # Handle missing order number
            final_response = "I can help with that. What is your order number?"

    # For other intents, just use the template
    else:
        final_response = response_template

    return final_response

In [9]:
# --- 1. Comment out the original interactive loop ---

# print("Customer Support Chatbot: Ask me anything! (Type 'exit' to quit)")
# while True:
#     user_input = input("You: ")
#     if user_input.lower() == 'exit':
#         break
#
#     answer = get_bot_response(user_input)
#     print(f"Bot: {answer}")


# --- 2. Add your test questions directly here ---

print("--- Chatbot Test Run ---")

# Test 1: Greeting
query1 = "Hello"
answer1 = get_bot_response(query1)
print(f"You: {query1}")
print(f"Bot: {answer1}\n")

# Test 2: Order Status (with order number)
query2 = "What is the status of my order #78901?"
answer2 = get_bot_response(query2)
print(f"You: {query2}")
print(f"Bot: {answer2}\n")

# Test 3: Order Status (without order number)
query3 = "Where is my package?"
answer3 = get_bot_response(query3)
print(f"You: {query3}")
print(f"Bot: {answer3}\n")

# Test 4: Return Policy
query4 = "How can I return an item?"
answer4 = get_bot_response(query4)
print(f"You: {query4}")
print(f"Bot: {answer4}\n")

# Test 5: Product Info
query5 = "Is this laptop waterproof?"
answer5 = get_bot_response(query5)
print(f"You: {query5}")
print(f"Bot: {answer5}\n")

# Test 6: Payment Issue
query6 = "My payment failed"
answer6 = get_bot_response(query6)
print(f"You: {query6}")
print(f"Bot: {answer6}\n")

# Test 7: Goodbye
query7 = "Okay, thanks"
answer7 = get_bot_response(query7)
print(f"You: {query7}")
print(f"Bot: {answer7}\n")

print("--- End of Test Run ---")

--- Chatbot Test Run ---
You: Hello
Bot: Hi! How can I help you today?

You: What is the status of my order #78901?
Bot: I'm sorry to hear that. Can you please provide your order number so I can check?

You: Where is my package?
Bot: I can help with that. What is your order number?

You: How can I return an item?
Bot: You can return products within 15 days via our online portal.

You: Is this laptop waterproof?
Bot: The 'Pro Laptop X' comes with an i7 processor, 16GB RAM, and a 1TB SSD.

You: My payment failed
Bot: We apologize for this error. Please contact our billing department at billing@shop.com with your order ID.

You: Okay, thanks
Bot: Goodbye! Have a nice day.

--- End of Test Run ---
